# Navigating OracleDB in Docker & Crafting a .NET 6 API!

Hey Tech Enthusiasts! 🌟 Join me live as I delve into the world of Docker and Oracle Database. We're setting up an OracleDB from scratch.

[twitch.tv/nullptrerrorasync/](https://www.twitch.tv/nullptrerrorasync/)

## Getting Started README
[Getting Started ./README.md](./README.md)

## Git

### Branching Strategy
1. `origin/main` Branch:
   * This is your production branch. Code here is always deployable. It reflects the current state of your production environment.
2. `origin/development` Branch:
   * A semi-stable branch where features and fixes are merged regularly.
   * This branch acts as a buffer for `origin/main`.
   * Regular builds and tests run against this branch.
3. `origin/feature/xyz` Branches:
   * For new features or non-critical fixes.
   * These are branched off `origin/development` and are merged back into `origin/development` once complete.
4. `origin/release/xyz` Branch:
   * When `origin/development` is ready for a **production release**, it's branched to a `origin/release/xyz` branch.
   * Any final tweaks or pre-release fixes happen here.
   * After successful testing, this branch is merged to main and tagged.
5. `origin/hotfix/xyz` Branches:
   * For critical fixes needed in production.
   * These are branched off `origin/main`, fixed, tested, and then merged back into both `origin/main` and `origin/development`.
6. `origin/environment/xyz` Branches:
   * To support multiple environments (like `origin/environment/webServer01` or `origin/environment/webServer02`), environment-specific branches can be created.
   * For example, there might be a `origin/environment/webServer01` branch. When you want to deploy to the `webServer01` environment, you merge `origin/main` into `origin/environment/webServer01`.

### Version Strategy with GitVersion

Use `Major`.`Minor`.`Patch` versioning with manual control over `Major` and `Minor` bumps.
Let Patch be automatic. This method relies less on commit messages or branch names.

1. `origin/main` Branch: Whenever a `origin/release/xyz` branch is merged into `origin/main`, it's tagged manually with the `origin/release/xyz` version (e.g., 2.3.0).
2. `origin/development` Branch: Inherits the version from `origin/main` but with an additional -alpha or -beta tag. So if main is on 2.3.0, `origin/development` might be 2.3.1-alpha.1. With each merge into `origin/development`, the number after -alpha. or -beta. increments.
3. `origin/feature/xyz` Branches: Inherits the version from `origin/development`. No unique versioning is required.
4. `origin/release/x.y.z` Branches: Once branched off `origin/development`, the -alpha or -beta tag is dropped. Any commits in this branch increase the Patch number. E.g., from 2.3.1 to 2.3.2.
5. `origin/hotfix/x.y.z` Branches: These branches will increment the Patch version from the current production version in `origin/main`.
6. Protection and Tagging
   * Protect important branches (like `origin/main` and `origin/environment/webdev01` branches) to prevent direct pushes. Instead, use merge requests (or pull requests in GitHub terms) to ensure that code is reviewed before being merged.
   * Use tags to mark specific points in the `origin/main` branch as production `origin/release/`, making it easy to track and revert to specific versions if necessary.

### Global Powershell Helpers

#### Global Helper Functions

In [1]:
# Make Assign and RunAndLog functions available to all scripts
function Assign([string]$VariableName, $Value) {
    Set-Variable -Scope Script -Name $VariableName -Value $Value
    if ($Value -eq $null -or $Value -eq '' -or $Value -match '^\s+$' -or $Value -match '^\t+$' -or $Value -match '^\n+$') { Write-Host "`$script:$VariableName = `$null" -ForegroundColor DarkGray }
    else { Write-Host "`$script:$VariableName = `$Value"; $Value }
}
function RunAndLog([System.Object[]]$ScriptBlocks) {
    $OriginalErrorActionPreference = $ErrorActionPreference
    foreach ($ScriptBlock in $ScriptBlocks) {
        $ErrorActionPreference = 'Continue'
        Write-Host "$(Get-Date -Format 'hh-mm-ss-fff-tt')>pwsh$($ScriptBlock.Code.ToString() -replace '\[', '［' -replace '\]', '］')" -ForegroundColor Cyan
        $outputData = @()
        $errorData = @()
        & $ScriptBlock.Code 2>&1 | ForEach-Object { if ($_ -is [System.Management.Automation.ErrorRecord]) { $errorData += $_ } else { $outputData += $_ } }
        $outputData | ForEach-Object { Write-Output $_ }
        $errorData | ForEach-Object {  if ($ScriptBlock.Required) {
            $ErrorActionPreference = 'Stop'
            throw $_;
        } else { Write-Output $_ } }
    }
    $ErrorActionPreference = $OriginalErrorActionPreference
}

#### Clean Project Up Functions

In [2]:
# Delete all local and remote branches and delete the .git folder for a clean start
Invoke-Command -ScriptBlock {
    RunAndLog -ScriptBlocks @(
        @{
            Code = { Assign 'branchesR' (git branch -r) }
            Required = $false
        },
        @{
            Code = { Assign 'branchesRFiltered' ($script:branchesR | Where-Object { $_ -match 'origin/([^ ]+)' } | ForEach-Object { $matches[1] }) }
            Required = $false
        },
        @{
            Code = { Assign 'branchesRFilteredDeleted' ($script:branchesRFiltered | ForEach-Object { git push origin --delete $_ -v }) }
            Required = $false
        },
        @{
            Code = { Assign 'branches' (git branch) }
            Required = $false
        },
        @{
            Code = { Assign 'branchesFiltered' ($script:branches | Where-Object { $_ -notmatch '^\*' }) }
            Required = $false
        },
        @{
            Code = { Assign 'branchesFilteredNames' ($script:branchesFiltered | ForEach-Object { $_.Trim() }) }
            Required = $false
        },
        @{
            Code = { Assign 'branchesFilteredDeleted' ($script:branchesFilteredNames | ForEach-Object { git branch -d $_ -v }) }
            Required = $false
        },
        @{
            Code = { Assign 'gitDir' ("$PWD\.git") }
            Required = $false
        }
        @{
            Code = { Assign 'gitDirDeleted' (Remove-Item -Path $script:gitDir -Recurse -Force -Verbose) }
            Required = $false
        },
        @{
            Code = { Assign 'gitIgnoreDir' ("$PWD\.gitignore") }
            Required = $false
        },
        @{
            Code = { Assign 'gitIgnoreDirDeleted' (Remove-Item -Path $script:gitIgnoreDir -Recurse -Force -Verbose) }
            Required = $false
        }
    )
}

07-49-51-625-PM>pwsh Assign 'branchesR' (git branch -r) 
$script:branchesR = $null
fatal: not a git repository (or any of the parent directories): .git
07-49-52-053-PM>pwsh Assign 'branchesRFiltered' ($script:branchesR | Where-Object { $_ -match 'origin/(［^ ］+)' } | ForEach-Object { $matches［1］ }) 
$script:branchesRFiltered = $null
07-49-52-057-PM>pwsh Assign 'branchesRFilteredDeleted' ($script:branchesRFiltered | ForEach-Object { git push origin --delete $_ -v }) 
$script:branchesRFilteredDeleted = $null
fatal: not a git repository (or any of the parent directories): .git
07-49-52-484-PM>pwsh Assign 'branches' (git branch) 
$script:branches = $null
fatal: not a git repository (or any of the parent directories): .git
07-49-52-997-PM>pwsh Assign 'branchesFiltered' ($script:branches | Where-Object { $_ -notmatch '^\*' }) 
$script:branchesFiltered = $null
07-49-52-998-PM>pwsh Assign 'branchesFilteredNames' ($script:branchesFiltered | ForEach-Object { $_.Trim() }) 
$script:branchesFiltered

### Initialize git repository with `git init .`

In [3]:
Invoke-Command -ScriptBlock {
    RunAndLog -ScriptBlocks @(
        @{
            Code = { Assign 'gitConfigInitDefaultBranch' (git config --global init.defaultBranch main) }
            Required = $true
        },
        @{
            Code = { Assign 'gitInit' (git init .) }
            Required = $true
        },
        @{
            Code = { Assign 'gitConfigCoreAutocrlf' (git config core.autocrlf true) }
            Required = $true
        }
    )
}

07-49-56-501-PM>pwsh Assign 'gitConfigInitDefaultBranch' (git config --global init.defaultBranch main) 
$script:gitConfigInitDefaultBranch = $null
07-49-57-015-PM>pwsh Assign 'gitInit' (git init .) 
$script:gitInit = $Value
Initialized empty Git repository in C:/LocalRepo/docker-images-oracle-database-single-instanance-12.1.0.2-ee/.git/
07-49-57-548-PM>pwsh Assign 'gitConfigCoreAutocrlf' (git config core.autocrlf true) 
$script:gitConfigCoreAutocrlf = $null


### Add the commit-msg git hook script to to define a standardregular expression for the commit message format

In [4]:
Invoke-Command -ScriptBlock {
    $script:hookScriptPath = "$PWD\.git\hooks\commit-msg"
    $script:commitMsgScript = @'
\#!/bin/sh
commit_msg_file=$1
commit_msg=$(cat "$commit_msg_file")
# Define your regular expression for the commit message format
commit_msg_regex='^(fix|feat|docs|test|devops|chore)\(.*\): .+$'
    
if ! echo "$commit_msg" | grep -Pq "$commit_msg_regex"; then
    echo "Error: Commit message format is invalid."
    echo "Commit Messages"
    echo "Commit messages should follow the Semantic Commit Messages format:"
    echo ""
    echo "label(namespace): title"
    echo ""
    echo "label is one of the following:"
    echo "  fix - Application bug fixes."
    echo "  feat - Application features."
    echo "  docs - changes to docs, e.g. docs(api.md): .. to change documentation."
    echo "  test - changes to Application tests infrastructure."
    echo "  devops - build-related work, e.g. CI related patches and general changes to the browser build infrastructure."
    echo "  chore - everything that doesn't fall under previous categories."
    echo "namespace is put in parenthesis after label and is optional. Must be lowercase."
    echo "title is a brief summary of changes."
    echo "description is optional, new-line separated from title and is in present tense."
    echo "footer is optional, new-line separated from description and contains 'fixes' / 'references' attribution to GitHub issues."
    exit 1
fi
'@ -replace '^\\{1}', ''
    RunAndLog -ScriptBlock @(
        @{
            Code = { Assign 'setContentOutput' (Set-Content -Path $script:hookScriptPath -Value $script:commitMsgScript) }
            Required = $true
        },
        @{
            Code = { Assign 'acl' (Get-Acl $script:hookScriptPath) }
            Required = $true
        },
        @{
            Code = { Assign 'currentUser' ([System.Security.Principal.WindowsIdentity]::GetCurrent().User) }
            Required = $true
        },
        @{
            Code = { Assign 'accessRule' (New-Object System.Security.AccessControl.FileSystemAccessRule($script:currentUser, [System.Security.AccessControl.FileSystemRights]::ReadAndExecute, [System.Security.AccessControl.AccessControlType]::Allow)) }
            Required = $true
        },
        @{
            Code = { Assign 'aclAddAccessRule' ($script:acl.AddAccessRule($script:accessRule)) }
            Required = $true
        },
        @{
            Code = { Assign 'setAclOutput' (Set-Acl -Path $script:hookScriptPath -AclObject $script:acl) }
            Required = $true
        }
    )
}

07-49-59-856-PM>pwsh Assign 'setContentOutput' (Set-Content -Path $script:hookScriptPath -Value $script:commitMsgScript) 
$script:setContentOutput = $null
07-49-59-867-PM>pwsh Assign 'acl' (Get-Acl $script:hookScriptPath) 
$script:acl = $Value

07-49-59-997-PM>pwsh Assign 'currentUser' (［System.Security.Principal.WindowsIdentity］::GetCurrent().User) 
$script:currentUser = $Value
07-50-00-010-PM>pwsh Assign 'accessRule' (New-Object System.Security.AccessControl.FileSystemAccessRule($script:currentUser, ［System.Security.AccessControl.FileSystemRights］::ReadAndExecute, ［System.Security.AccessControl.AccessControlType］::Allow)) 
$script:accessRule = $Value
07-50-00-020-PM>pwsh Assign 'aclAddAccessRule' ($script:acl.AddAccessRule($script:accessRule)) 
$script:aclAddAccessRule = $null
07-50-00-022-PM>pwsh Assign 'setAclOutput' (Set-Acl -Path $script:hookScriptPath -AclObject $script:acl) 
$script:setAclOutput = $null
    Directory: C:\LocalRepo\docker-images-oracle-database-single-instanance

### Add VisualStudio.gitignore

In [5]:
Invoke-Command -ScriptBlock {
    $script:gitignoreUrl = "https://raw.githubusercontent.com/github/gitignore/main/VisualStudio.gitignore"
    $script:savePath = "$PWD\.gitignore"

    RunAndLog -ScriptBlocks @(
        @{
            Code = { Assign 'gitignoreContent' (Invoke-WebRequest -Uri $script:gitignoreUrl -ErrorAction Stop).Content }
            Required = $true
        },
        @{
            Code = { Assign 'setContentOutput' (Set-Content -Path $script:savePath -Value $script:gitignoreContent -Force -Encoding utf8) }
            Required = $true
        }
    )
}

07-50-03-774-PM>pwsh Assign 'gitignoreContent' (Invoke-WebRequest -Uri $script:gitignoreUrl -ErrorAction Stop).Content 


$script:gitignoreContent = $Value
## Ignore Visual Studio temporary files, build results, and
## files generated by popular Visual Studio add-ons.
##
## Get latest from https://github.com/github/gitignore/blob/main/VisualStudio.gitignore

# User-specific files
*.rsuser
*.suo
*.user
*.userosscache
*.sln.docstates

# User-specific files (MonoDevelop/Xamarin Studio)
*.userprefs

# Mono auto generated files
mono_crash.*

# Build results
[Dd]ebug/
[Dd]ebugPublic/
[Rr]elease/
[Rr]eleases/
x64/
x86/
[Ww][Ii][Nn]32/
[Aa][Rr][Mm]/
[Aa][Rr][Mm]64/
bld/
[Bb]in/
[Oo]bj/
[Ll]og/
[Ll]ogs/

# Visual Studio 2015/2017 cache/options directory
.vs/
# Uncomment if you have tasks that create the project's static files in wwwroot
#wwwroot/

# Visual Studio 2017 auto generated files
Generated\ Files/

# MSTest test Results
[Tt]est[Rr]esult*/
[Bb]uild[Ll]og.*

# NUnit
*.VisualState.xml
TestResult.xml
nunit-*.xml

# Build Results of an ATL Project
[Dd]ebugPS/
[Rr]eleasePS/
dlldata.c

# Benchmark Results
Benchm

### Git add files

In [ ]:
Invoke-Command -ScriptBlock {
    RunAndLog -ScriptBlocks @(
        @{
            Code = { Assign 'gitAdd' (git add .) }
            Required = $true
        }
    )
}

### Commit main

In [ ]:
Invoke-Command -ScriptBlock {
    RunAndLog -ScriptBlocks @(
        @{
            Code = { Assign 'commitMain' (git commit -m "docs(README.md): initial commit") }
            Required = $true
        }
    )
}

In [ ]:
Invoke-Command -ScriptBlock {
    RunAndLog -ScriptBlocks @(
        @{
            Code = { (git add .) }
            Required = $true
        },
        @{
            Code = { Assign 'commitMainOutput' (git commit -m "docs(README.md): initial commit output") }
            Required = $true
        }
    )
}

### Initialize Git Flow

In [ ]:
Invoke-Command -ScriptBlock {
    RunAndLog -ScriptBlocks @(
        @{
            Code = { Assign 'gitFlowInitDF' (git flow init -d -f) }
            Required = $true
        }
    )
}

### Add Origin

In [ ]:
Invoke-Command -ScriptBlock {
    $script:gitUrl = 'URL_HERE'
    RunAndLog -ScriptBlock @(
        @{
            Code = { Assign 'gitRemoteAddOrigin' (git remote add origin $script:gitUrl) }
            Required = $true
        }
    )
}